In [1]:
import sys
import os
current_path = os.getcwd()  # 获取当前工作目录
print("当前路径:", current_path)
current_path = current_path.replace('/samples', '')
print(sys.path)  # 显示所有模块搜索路径
sys.path.append(current_path)  # 添加自定义路径
print(sys.path)  # 显示所有模块搜索路径

当前路径: /Users/yang/PycharmProjects/Option/samples
['/Users/yang/PycharmProjects/Option/samples', '/Users/yang/PycharmProjects/Option', '/opt/anaconda3/lib/python312.zip', '/opt/anaconda3/lib/python3.12', '/opt/anaconda3/lib/python3.12/lib-dynload', '', '/opt/anaconda3/lib/python3.12/site-packages', '/opt/anaconda3/lib/python3.12/site-packages/aeosa']
['/Users/yang/PycharmProjects/Option/samples', '/Users/yang/PycharmProjects/Option', '/opt/anaconda3/lib/python312.zip', '/opt/anaconda3/lib/python3.12', '/opt/anaconda3/lib/python3.12/lib-dynload', '', '/opt/anaconda3/lib/python3.12/site-packages', '/opt/anaconda3/lib/python3.12/site-packages/aeosa', '/Users/yang/PycharmProjects/Option']


QuantLib 期权定价示例
#### 本示例展示如何使用`QlCalendar`、`QlStocks`和`QlVanillaOptions`类创建和定价多种期权，并测试不同定价引擎的功能。


In [2]:
import QuantLib as ql
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from src.QlCalendar import QlCalendar
from src.QlStocks import QlStocks
from src.QlVanillaOptions import QlVanillaOptions

## 1. 初始化设置
#### 创建日历、股票和期权对象

In [3]:
# 设置初始日期
start_date = ql.Date(31, 5, 2025)  # 2025年5月31日
ql_calendar = QlCalendar(init_date=start_date)

# 创建股票对象
s = QlStocks(ql_calendar)
s.black_scholes(100.0)  # 创建一只股票，价格为100

# 创建期权对象
options = QlVanillaOptions(s.df)  # 使用第一只股票创建期权


today:  June 2nd, 2025


## 2. 测试不同定价引擎
### 2.1 AnalyticEuropeanEngine (解析解)

In [4]:
# 创建单个欧式看跌期权
op = options.european_option(
    'put',
    100,  # 行权价
    ql.Date(30, 6, 2025),  # 到期日
    qlEngine=ql.AnalyticEuropeanEngine
)

# 计算并显示结果
df = options.NPV(op)
print("AnalyticEuropeanEngine 结果:")
print(df[['strike_prices', 'maturity_dates', 'types', 'NPV']])


AnalyticEuropeanEngine 结果:
   strike_prices   maturity_dates types       NPV
0            100  June 30th, 2025   put  2.050577


### 2.2 MCEuropeanEngine (蒙特卡洛模拟)

In [5]:
# 创建多个欧式看涨期权
strikes = np.linspace(90, 110, 5)  # 5个不同行权价
maturities = [ql.Date(30, 6, 2025)] * len(strikes)  # 相同到期日
types = ['call'] * len(strikes)  # 全部为看涨期权

op = options.european_option(
    types,
    strikes,
    maturities,
    qlEngine=ql.MCEuropeanEngine,
    traits = "pseudorandom", # could use "lowdiscrepancy"
    timeSteps=20,  # 时间步数
    requiredSamples=1000,  # 路径数量
    seed=42  # 随机种子
)

# 计算并显示结果
df = options.NPV(op)
print("\nMCEuropeanEngine 结果:")
print(df[['strike_prices', 'maturity_dates', 'types', 'NPV']])



MCEuropeanEngine 结果:
   strike_prices   maturity_dates types        NPV
0           90.0  June 30th, 2025  call  10.103704
1           95.0  June 30th, 2025  call   5.552839
2          100.0  June 30th, 2025  call   2.234077
3          105.0  June 30th, 2025  call   0.579401
4          110.0  June 30th, 2025  call   0.079597


### 2.3 FdBlackScholesVanillaEngine (有限差分法)

In [6]:
# 创建多个欧式看跌期权
strikes = np.linspace(80, 120, 5)  # 5个不同行权价
maturities = [ql.Date(31, 12, 2025)] * len(strikes)  # 相同到期日
types = ['put'] * len(strikes)  # 全部为看跌期权

tGrid, xGrid = 2000, 200
op = options.european_option(
    types,
    strikes,
    maturities,
    ql.EuropeanExercise,
    ql.FdBlackScholesVanillaEngine,
    tGrid,
    xGrid
)

# 计算并显示结果
df = options.NPV(op)
print("\nFdBlackScholesVanillaEngine 结果:")
print(df[['strike_prices', 'maturity_dates', 'types', 'NPV']])



FdBlackScholesVanillaEngine 结果:
   strike_prices       maturity_dates types        NPV
0           80.0  December 31st, 2025   put   0.272321
1           90.0  December 31st, 2025   put   1.473684
2          100.0  December 31st, 2025   put   4.661148
3          110.0  December 31st, 2025   put  10.291280
4          120.0  December 31st, 2025   put  17.923437


### 2.4 MCAmericanEngine (美式期权蒙特卡洛)

In [7]:
# 创建多个美式看涨期权
# strikes = np.linspace(95, 105, 3)  # 3个不同行权价
# maturities = [ql.Date(31, 12, 2025)] * len(strikes)  # 相同到期日
# types = ['call'] * len(strikes)  # 全部为看涨期权
# 
# op = options.european_option(
#     types,
#     strikes,
#     maturities,
#     qlExercise=ql.AmericanExercise,
#     qlEngine=ql.MCAmericanEngine,
#     timeSteps = 200,
#     traits = "pseudorandom", # could use "lowdiscrepancy"
#     requiredSamples=1000,  # 路径数量
#     seed=42  # 随机种子
# )
# 
# # 计算并显示结果
# df = options.NPV(op)
# print("\nMCAmericanEngine 结果:")
# print(df[['strike_prices', 'maturity_dates', 'types', 'NPV']])
# 
# # %% md
# ## 3. 测试风险指标计算
# #### 计算并显示所有期权的希腊字母
# # %%
# # 计算delta
# df_delta = options.delta()
# print("\nDelta 值:")
# print(df_delta[['strike_prices', 'types', 'delta']])
# 
# # 计算gamma
# df_gamma = options.gamma()
# print("\nGamma 值:")
# print(df_gamma[['strike_prices', 'types', 'gamma']])
# 
# # 计算vega
# df_vega = options.vega()
# print("\nVega 值:")
# print(df_vega[['strike_prices', 'types', 'vega']])
# 
# # %% md
# ## 4. 批量创建大量期权
# #### 测试5000个期权的批量创建和定价
# # %%
# # 生成随机行权价
# prices = np.random.normal(loc=100, scale=10, size=5000)
# 
# # 创建5000个看涨期权
# options.european_option(
#     'call',
#     prices,
#     ql.Date(31, 12, 2025),
#     qlEngine=ql.AnalyticEuropeanEngine
# )
# 
# # 计算并显示部分结果
# df = options.NPV()
# print("\n5000个期权的部分结果:")
# print(df.head())  # 只显示前5行
# 
# # 计算并显示统计信息
# print("\n5000个期权的统计信息:")
# print(df['NPV'].describe())